# **Feature engeneering**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pycountry_convert as pc
import os

os.chdir(os.path.dirname(os.getcwd()))      # Move to parent directory

from utils import compute_difficulty_index, compute_bmi



In [ ]:
print(os.getcwd())

# Dataset download
cyclists_df = pd.read_csv('dataset/cyclists.csv')
races_df = pd.read_csv('dataset/races.csv')

#### ```categorical_height``` column


Since cycling is a sport in which, on average for the same height, the weight is similar, we group height by category with 5cm intervals to look for possible outliers in weight using conditional boxplots.

In [ ]:
# Define the bins for the height category and create the feature
bins = [154, 159, 164, 169, 174, 179, 184, 189, 194, 199, 204]
labels = list(range(10))

cyclists_df['categorical_height'] = pd.cut(cyclists_df['height'], bins=bins, labels=labels, right=False, include_lowest=True)

print(cyclists_df[['height', 'categorical_height']].head(10))

Conditional Box Plot

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x='categorical_height', y='weight', data=cyclists_df)
plt.title('Boxplot of Weight Conditioned on Height Category')
plt.xlabel('Height Category')
plt.ylabel('Weight')

# Set y-axis intervals
plt.yticks(np.arange(50, cyclists_df['weight'].max() + 5, 5))

plt.show()

We show the number of cyclists in each defined category (excluding those with zero weight) to evaluate the significance of the outliers identified from the boxplots.

In [ ]:
# Exclude cyclists with null weight
cyclists_df_non_null_weight = cyclists_df[cyclists_df['weight'].notnull()]

# Count the number of cyclists in each height category
categorical_height_counts = cyclists_df_non_null_weight['categorical_height'].value_counts().sort_index()

# Plot the results
plt.figure(figsize=(8, 4))
ax = categorical_height_counts.plot(kind='bar')
plt.title('Number of Cyclists per Height Category (Excluding Null Weights)')
plt.xlabel('Height Category')
plt.ylabel('Number of Cyclists')

# Add the count above each bin
for i, count in enumerate(categorical_height_counts):
    ax.text(i, count + 5, str(count), ha='center', va='bottom')

# Rotate x-axis labels to horizontal
plt.xticks(rotation=0)

plt.show()

#### ```BMI_idx``` column

In [ ]:
cyclists_df['BMI_idx'] = cyclists_df.apply(
    lambda row: compute_bmi(row['weight'], row['height']),
    axis=1
)

print(cyclists_df[['weight', 'height', 'BMI_idx']].head(10))

Categorizing BMI index

In [ ]:
# Define the BMI categories and their corresponding labels
bmi_bins = [0, 18.5, 24.9, 29.9, np.inf]
bmi_labels = ['Underweight', 'Normal weight', 'Overweight', 'Obese']

# Create the 'BMI' categorical column
cyclists_df['BMI'] = pd.cut(cyclists_df['BMI_idx'], bins=bmi_bins, labels=bmi_labels, right=False, include_lowest=True)

# Display the first few rows to verify
print(cyclists_df[['_url', 'BMI_idx', 'BMI']].head(10))

In [ ]:
# Count the number of cyclists per BMI category
bmi_category_counts = cyclists_df['BMI'].value_counts().sort_index()

# Plot the results
plt.figure(figsize=(8, 4))
ax = bmi_category_counts.plot(kind='bar')
plt.title('Number of Cyclists per BMI Category')
plt.xlabel('BMI Category')
plt.ylabel('Number of Cyclists')

# Add the count above each bin
for i, count in enumerate(bmi_category_counts):
    ax.text(i, count + 5, str(count), ha='center', va='bottom')

# Rotate x-axis labels to horizontal
plt.xticks(rotation=0)

plt.show()

Null values for BMI index

In [ ]:
null_bmi_idx_count = cyclists_df['BMI_idx'].isnull().sum()
print(f"Number of null values in BMI_idx column: {null_bmi_idx_count}")

As expected the most of the cyclists are normal weight

In [ ]:
cyclists_df = cyclists_df.drop(columns=['BMI_idx'])
# Create a pairplot excluding the 'categorical_height' column
sns.pairplot(cyclists_df.drop(columns=[]), hue='BMI')
plt.show()

#### ```continent``` column

In [ ]:
# Function to convert country name to continent
def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        return 'Unknown'

# Apply the function to create a new column 'continent'
cyclists_df['continent'] = cyclists_df['nationality'].apply(country_to_continent)

# Display the first few rows to verify
print(cyclists_df[['nationality', 'continent']].head(10))

In [ ]:
# Count the number of cyclists per continent
continent_counts = cyclists_df['continent'].value_counts()

# Plot the number of cyclists per continent as a histogram
plt.figure(figsize=(8, 5))
ax = continent_counts.plot(kind='bar')
plt.title('Number of Cyclists per Continent')
plt.xlabel('Continent')
plt.ylabel('Number of Cyclists')
plt.xticks(rotation=45)

# Add the count above each bin
for i, count in enumerate(continent_counts):
    ax.text(i, count + 50, str(count), ha='center', va='bottom')

plt.show()

In [ ]:
# Create a pairplot excluding the 'categorical_height' column
sns.pairplot(cyclists_df.drop(columns=['categorical_height']), hue='continent')
plt.show()

Given that majority of cyclists are european let's deep into european cyclist analysis

In [ ]:
# Filter cyclists by European continent
european_df = cyclists_df[cyclists_df['continent'] == 'Europe'].drop(columns=['continent'])

In [ ]:
# Count the number of cyclists per nationality
nationality_counts = european_df['nationality'].value_counts()

# Plot the number of cyclists per nationality as a histogram
plt.figure(figsize=(14, 7))
ax = nationality_counts.plot(kind='bar')
plt.title('Number of Cyclists per Nationality')
plt.xlabel('Nationality')
plt.ylabel('Number of Cyclists')
plt.xticks(rotation=90)

# Add the count above each bin
for i, count in enumerate(nationality_counts):
    ax.text(i, count + 10, str(count), ha='center', va='bottom')

plt.show()

In [ ]:
# Create a pairplot excluding the 'categorical_height' column
sns.pairplot(european_df.drop(columns=['categorical_height']), hue='nationality')
plt.show()

#### ```cyclist_win``` column

Count how many stages won each cyclist

In [ ]:
# Group by cyclist and count the number of times position is 0
stages_won = races_df[races_df['position'] == 0].groupby('cyclist').size().reset_index(name='position')

# Add cyclists who never had position 0 with counter set to 0
all_cyclists = pd.DataFrame(races_df['cyclist'].unique(), columns=['cyclist'])
stages_won_df = pd.merge(all_cyclists, stages_won, on='cyclist', how='left').fillna(0)

# Ensure position is an integer
stages_won_df['position'] = stages_won_df['position'].astype(int)
# Merge the stages_won_df with cyclists_df to add the 'cyclist_win' column
cyclists_df = cyclists_df.merge(stages_won_df, left_on='_url', right_on='cyclist', how='left')

# Rename the 'position' column to 'cyclist_win'
cyclists_df = cyclists_df.rename(columns={'position': 'cyclist_win'})

# Drop the redundant 'cyclist' column
cyclists_df = cyclists_df.drop(columns=['cyclist'])

print(cyclists_df[["_url", "cyclist_win"]].head(3))

#check correctness of the computation (sum(races won) = num. of races)
print("Total cyclist_wins:", cyclists_df['cyclist_win'].sum())


Some useful analysis on win column like max, min, counting...

In [ ]:
cyclists_df['cyclist_win'].describe()

Plot distribution of stages won

In [ ]:
# Plot the distribution of stages won
plt.figure(figsize=(8, 4))
sns.histplot(cyclists_df['cyclist_win'], bins=range(0, int(cyclists_df['cyclist_win'].max()) + 2), kde=False)
plt.title('Distribution of Stages Won for each Cyclist')
plt.xlabel('Number of Stages Won')
plt.ylabel('Number of Cyclists')
plt.show()

Scatter plot

In [ ]:
# Create a pairplot with 'cyclist_win' as the hue
sns.pairplot(cyclists_df.drop(columns=['categorical_height']), hue='cyclist_win')
plt.show()

#### ```team_win``` column

In [ ]:
# Group by team and count the number of times position is 0
stages_won_by_team = races_df[races_df['position'] == 0].groupby('cyclist_team').size().reset_index(name='position')

# Add teams who never had position 0 with counter set to 0
all_teams = pd.DataFrame(races_df['cyclist_team'].unique(), columns=['cyclist_team'])
stages_won_team_df = pd.merge(all_teams, stages_won_by_team, on='cyclist_team', how='left').fillna(0)

# Ensure position is an integer
stages_won_team_df['position'] = stages_won_team_df['position'].astype(int)

# Rename the 'position' column to 'team_win'
stages_won_team_df = stages_won_team_df.rename(columns={'position': 'team_win'})

# Merge the stages_won_team_df with races_df to add the 'team_win' column
races_df = races_df.merge(stages_won_team_df, on='cyclist_team', how='left')

print(races_df[['cyclist_team', 'team_win']].head(10))


In [ ]:
print(f"Null values in team_win column: {int(races_df['team_win'].isnull().sum())}")

In [ ]:
races_df['team_win'].describe()

Distribution Plot

In [ ]:
# Plot the number of races won per team as a histogram
plt.figure(figsize=(16, 7))
ax = stages_won_team_df.set_index('cyclist_team')['team_win'].plot(kind='bar')
plt.title('Number of Races Won per Team')
plt.xlabel('Team')
plt.ylabel('Number of Races Won')
plt.xticks(rotation=90)

# Add the count above each bin
for i, count in enumerate(stages_won_team_df['team_win']):
    ax.text(i, count + 1, str(count), ha='center', va='bottom')

plt.show()

#### ```difficulty_idx``` column

In [ ]:
races_df['difficulty_idx'] = races_df.apply(
        lambda row: compute_difficulty_index(row['length']/1000, row['climb_total']/1000, row['profile']),
        axis=1
        )

In [ ]:
print(f"Null values in avg_slope column: {int(races_df['difficulty_idx'].isnull().sum())}")

In [ ]:
races_df["difficulty_idx"].describe()

Normalize this difficulty index

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the 'difficulty_index' column
races_df['difficulty_index_normalized'] = scaler.fit_transform(races_df[['difficulty_idx']]).round(2)
races_df.drop(columns=['difficulty_idx'], inplace=True)
races_df.rename(columns={'difficulty_index_normalized': 'difficulty_idx'}, inplace=True)

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y='difficulty_idx', data=races_df)
plt.title('Boxplot of Difficulty Index')
plt.ylabel('Difficulty Index')
plt.show()

In [ ]:
# Plot the distribution of the difficulty index
plt.figure(figsize=(8, 4))
sns.histplot(races_df['difficulty_idx'], bins=30)
plt.title('Distribution of Difficulty Index')
plt.xlabel('Difficulty Index')
plt.ylabel('Frequency')
plt.show()

##### ```min_duration``` column 

We decided to drop the ```delta``` column and the ```duration``` column as they contain many errors and as in the final clustering we will try to cluster stages and riders rather than riders and stages together. For this reason, we create a new feature min_duration that contains the total time taken for the first cyclist to finish the stage, as this could be another measure to assess the difficulty of the stage.

In [ ]:
# DROP THIS CODE WHEN THE PROJECT IS FINISHED: this split is already done in DU

# Split the 'date' column into 'start_date' and 'duration'
races_df['date'] = pd.to_datetime(races_df['date'])
races_df['start_date'] = races_df['date'].dt.date
races_df['duration'] = races_df['date'].dt.time

In [ ]:
# Group by '_url' and take the first entry in 'duration' for each group
races_df['min_duration'] = races_df.groupby('_url')['duration'].transform('first')

# Display the first few rows to verify
print(races_df[['duration', 'min_duration']].head(7))

# Convert 'min_duration' to total seconds
races_df['min_duration'] = races_df['min_duration'].apply(lambda x: int(pd.to_timedelta(str(x)).total_seconds()))

# Display the first few rows to verify
print(races_df[['min_duration']].head(7))

In [ ]:
races_df['min_duration'].describe()

In [ ]:
grouped_df = races_df.groupby('min_duration')[['length', '_url']].value_counts().sort_index()
print(grouped_df)

### ```season``` column 

In [ ]:
# Split the 'date' column into 'start_date' and 'duration'
races_df['start_date'] = pd.to_datetime(races_df['date']).dt.date

In [ ]:

def get_season(date):
    month = date.month
    day = date.day

    if (month == 12 and day >= 21) or (month <= 2) or (month == 3 and day < 20):
        return 'Winter'
    elif (month == 3 and day >= 20) or (month <= 5) or (month == 6 and day < 21):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month <= 8) or (month == 9 and day < 22):
        return 'Summer'
    else:
        return 'Autumn'

# Apply the custom get_season function to create a new column 'season'
races_df['season'] = races_df['start_date'].apply(lambda x: get_season(pd.to_datetime(x)))

# Display the first few rows to verify
print(races_df[['start_date', 'season']].head(5))

### ```cyclist_level``` column in races_df

The level of the cyclist is relative to the period of the race considered 

In [ ]:
# Sort the `races_df` DataFrame by the '_url' and 'start_date' columns 
races_df = races_df.sort_values(by=['_url', 'start_date'], kind='mergesort')  

**position weigth**

In [ ]:
WEIGHTS = {
    'win_points': 1.0,           # Weight for wins
    'second_place_points': 0.95, # Weight for second places
    'third_place_points': 0.90,  # Weight for third places
    'fourth_place_points': 0.85, # Weight for fourth places
}

### **Formula for Calculating Cyclist Level**

Let:
- $ V $ = Normalized Level of the cyclist
- $ W_i $ = Weight for each type of placement (i.e., wins, second places, third places, fourth places)
- $ N_i $ = Number of placements of type $ i $ 
- $ T $ = Total number of races completed by the cyclist
- $ P_{\text{i}} $ = points of the reaces where cyslit arrived i-th

The formula for the normalized level $ V $ can be expressed as:

$$
V = \frac{1}{T} \sum_{i=1}^{4} \left( N_i \cdot W_i \cdot P_i \right)
$$

Where:
- \( i \) corresponds to the different categories:
  - \( P_1 \) = Number of wins (weighted by \( W_1 = 5 \))
  - \( P_2 \) = Number of second places (weighted by \( W_2 = 3 \))
  - \( P_3 \) = Number of third places (weighted by \( W_3 = 2 \))
  - \( P_4 \) = Number of fourth places (weighted by \( W_4 = 1 \))

**OBS:**  in the case where this column is added in the races_df dataset all data are relative up to the *start_date* of the row in conisderation. In the case where the column is added in the cyclist_df all races done by the cyclist are taken, thus conisdering his whole career


In [ ]:
import bisect

# Initialize the dictionary
cyclist_performance = {}

# iteration in all races to compute the cyclist level
for _, row in races_df.iterrows():

    cyclist = row['cyclist']
    position = row['position']
    points = row['points']

    # Initialize the nested dictionary if the cyclist is not already in the dictionary
    if cyclist not in cyclist_performance:
        cyclist_performance[cyclist] = {
            'win_points': [],
            'second_place_points': [],
            'third_place_points': [],
            'fourth_place_points': [],
            'total_races': 0
        }

    cyclist_performance[cyclist]['total_races'] += 1
    
    # Add the date and points as a tuple to the appropriate list based on the position
    if position == 0:
        cyclist_performance[cyclist]['win_points'].append(points)
    elif position == 1:
        cyclist_performance[cyclist]['second_place_points'].append(points)
    elif position == 2:
        cyclist_performance[cyclist]['third_place_points'].append(points)
    elif position == 3:
        cyclist_performance[cyclist]['fourth_place_points'].append(points)
    

    normalized_level = 0.0

    # Sum of placements weighted by their position and race score before the date
    placement_sum = 0
    for position, weight in WEIGHTS.items():
        for points in cyclist_performance[cyclist].get(position, []):
            placement_sum += weight * points
        
        normalized_level = placement_sum / cyclist_performance[cyclist]['total_races']
    
    # Update the 'cyclist_level' column for the current row
    races_df.at[row.name, 'cyclist_level'] = normalized_level


In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(races_df['cyclist_level'], bins=100, kde=True)
plt.title('Distribution of Cyclist Level')
plt.xlabel('Cyclist Level')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(y='cyclist_level', data=races_df)
plt.title('Boxplot of Cyclist Level')
plt.ylabel('Cyclist Level')
plt.show()

### ```cyclist_level``` column in cyclists_df

cyclist's level is calculated by considering all his races. 
The formula for calculating the level is the same as shown above: 
$$
V = \frac{1}{T} \sum_{i=1}^{4} \left( N_i \cdot W_i \cdot P_i \right)
$$

In [ ]:

# add a cyclist_level column to the cyclists_df
cyclists_df['cyclist_level'] = 0.0

arr = ['win_points', 'second_place_points', 'third_place_points', 'fourth_place_points', 'total_races']

for cyclist, races in cyclist_performance.items():
        placement_sum = 0

        win_points = cyclist_performance[cyclist]['win_points']
        second_points = cyclist_performance[cyclist]['second_place_points']
        third_points = cyclist_performance[cyclist]['third_place_points']
        fourth_points = cyclist_performance[cyclist]['fourth_place_points']
        total_races = cyclist_performance[cyclist]['total_races']

        for points in win_points:
            placement_sum += WEIGHTS['win_points'] * points
        for points in second_points:
            placement_sum += WEIGHTS['second_place_points'] * points
        for points in third_points:
            placement_sum += WEIGHTS['third_place_points'] * points
        for points in fourth_points:
            placement_sum += WEIGHTS['fourth_place_points'] * points
        
        normalized_level = placement_sum+total_races

        # Update the 'cyclist_level' in the cyclist_df
        cyclists_df.loc[cyclists_df['_url'] == cyclist, 'cyclist_level'] = normalized_level

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(cyclists_df['cyclist_level'], bins=110, kde=True)
plt.title('Distribution of Cyclist Level')
plt.xlabel('Cyclist Level')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(y='cyclist_level', data=cyclists_df)
plt.title('Boxplot of Cyclist Level')
plt.ylabel('Cyclist Level')
plt.show()

#### ```avg_slope``` column

Average slope of the stage can be useful as another indicator of stage complexity. We expect that the higher the slope the more complex is the race. 

In [ ]:
# Calculate the average slope and create the new column
races_df['avg_slope'] = ((races_df['climb_total'] / races_df['length'])*100).round(4)

# Display the first few rows to verify
print(races_df[['climb_total', 'length', 'avg_slope']].head(10))

In [ ]:
print(f"Null values in avg_slope column: {int(races_df['avg_slope'].isnull().sum())}")

In [ ]:
races_df['avg_slope'].describe()

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y='avg_slope', data=races_df)
plt.title('Boxplot of Average Slope')
plt.ylabel('Average Slope')
plt.show()

In [ ]:
# Plot the distribution of the average slope
plt.figure(figsize=(8, 4))
sns.histplot(races_df['avg_slope'], bins=30)
plt.title('Distribution of Average Slope')
plt.xlabel('Average Slope')
plt.ylabel('Frequency')
plt.show()

#### ```strength_idx``` column

The <b>Cyclist's  Strength</b> Indicator would essentially represent each cyclist's typical competitiveness level by averaging the startlist_quality scores across all stages they participated in. This feature can give insight into a cyclist's experience and the overall competitiveness of the races they've been involved in.

In [ ]:
# compute the average startlist quality for each cyclist
cyclist_strength = races_df.groupby('cyclist')['startlist_quality'].mean().reset_index()
cyclist_strength.rename(columns={'startlist_quality': 'strength_idx'}, inplace=True)

cyclists_df = cyclists_df.merge(cyclist_strength, left_on='_url', right_on='cyclist', how='left')


In [ ]:
print(f"Null values in strength_idx column: {int(cyclists_df['strength_idx'].isnull().sum())}")

In [ ]:
cyclists_df['strength_idx'].describe()

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y='strength_idx', data=cyclists_df)
plt.title('Boxplot of Strength Index')
plt.ylabel('Strength Index')
plt.show()

In [ ]:
# Plot the distribution of the strength index
plt.figure(figsize=(8, 4))
sns.histplot(cyclists_df['strength_idx'], bins=30)
plt.title('Distribution of Strength Index')
plt.xlabel('Strength Index')
plt.ylabel('Frequency')
plt.show()

#### ```best_position``` column

Given that many of them never won a stage, this column represents the best position achieved by a rider in a stage.

In [ ]:
best_finish_position = races_df.groupby('cyclist')['position'].min().reset_index()

# Step 2: Rename the column for clarity
best_finish_position.rename(columns={'position': 'best_position'}, inplace=True)

# Step 3: Merge the result back to the original DataFrame
cyclists_df = cyclists_df.merge(best_finish_position, left_on='_url', right_on='cyclist', how='left')

cyclists_df.drop(columns=['cyclist_x'], inplace=True)

In [ ]:
print(f"Null values in best_position column: {int(cyclists_df['best_position'].isnull().sum())}")

In [ ]:
cyclists_df['best_position'].describe()

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y='best_position', data=cyclists_df)
plt.title('Boxplot of Best Position Reached by Cyclist')
plt.ylabel('Best Position')
plt.show()

In [ ]:
# Plot the distribution of the strength index
plt.figure(figsize=(8, 4))
sns.histplot(cyclists_df['best_position'], bins=30)
plt.title('Distribution of Best Position Reached by Cyclist')
plt.xlabel('Best Position')
plt.ylabel('Frequency')
plt.show()

Checking the number of races in which cyclists who have never finished above 150th position have participated, with the assumption that these are cyclists with participation limited to a few stages or a single stage.

In [ ]:
# Filter cyclists who have never finished above 150th position
cyclists_never_above_150 = cyclists_df[cyclists_df['best_position'] > 150]['_url']

# Filter races_df to include only these cyclists
races_never_above_150 = races_df[races_df['cyclist'].isin(cyclists_never_above_150)]

# Count the number of stages each of these cyclists has participated in
# NOTE: at this point no cyclist has a duplicated entry in the same stage
races_count_never_above_150 = races_never_above_150['cyclist'].value_counts()

# Merge the counts with the best positions
races_count_with_best_position = races_count_never_above_150.to_frame().merge(
    cyclists_df, left_index=True, right_on='_url',
).rename(columns={'_url': 'cyclist name', 'count': 'Number of Stages', 'best_position': 'Best Position'})

# Display the result
print(races_count_with_best_position[['cyclist name', 'Number of Stages', 'Best Position']])

As expected, these cyclists will take part in very few or only one stage.

#### ```best_position_std``` column

Given that many of them never won a stage, this column represents the best position achieved by a rider in a stage.

In [ ]:
best_finish_position = races_df.groupby('cyclist')['position'].std().reset_index()

# Step 2: Rename the column for clarity
best_finish_position.rename(columns={'position': 'best_position_std'}, inplace=True)

# Step 3: Merge the result back to the original DataFrame
cyclists_df = cyclists_df.merge(best_finish_position, left_on='_url', right_on='cyclist', how='left')

In [ ]:
print(f"Null values in best_position_std column: {int(cyclists_df['best_position_std'].isnull().sum())}")

In [ ]:
cyclists_df['best_position_std'].describe()

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y='best_position_std', data=cyclists_df)
plt.title('Boxplot of Best Position Standard Deviation Column')
plt.ylabel('Best Position Standard Deviation')
plt.show()

In [ ]:
# Plot the distribution of the strength index
plt.figure(figsize=(8, 4))
sns.histplot(cyclists_df['best_position_std'], bins=30)
plt.title('Distribution of Best Position Standard Deviation Column')
plt.xlabel('Best Position Standard Deviation')
plt.ylabel('Frequency')
plt.show()

TODO: comment what this results means, why high std? this probably means inconsistency trought time or similar things

#### ```cyclist_experience``` column

Count how many stages the cyclist partecipate as an overall experience

In [ ]:
# Group by cyclist and count the number of stages each cyclist participated in 
# Note: aassuming no duplicates in same stage
stages_participated = races_df.groupby('cyclist').size().reset_index(name='cyclist_experience')

# Rename 'cyclist' in stages_participated to avoid conflicts
stages_participated = stages_participated.rename(columns={'cyclist': 'cyclist_id'})

# Merge the stages_participated with cyclists_df to add the 'cyclist_experience' column
# Specify suffixes to avoid duplicate columns
cyclists_df = cyclists_df.merge(stages_participated, left_on='_url', right_on='cyclist_id', how='left')

# Drop the redundant 'cyclist_id' column from the merged DataFrame
cyclists_df = cyclists_df.drop(columns=['cyclist_id'])

# Replace NaN values in cyclist_experience with 0 (for cyclists with no participation records)
cyclists_df['cyclist_experience'] = cyclists_df['cyclist_experience'].fillna(0).astype(int)

# Check the output
print(cyclists_df[["_url", "cyclist_experience"]].head(3))

# Verify correctness of the computation (sum of cyclist_experience should match number of rows in races_df)
print("Total cyclist_experience:", cyclists_df['cyclist_experience'].sum())

cyclists_df.drop(columns=['cyclist_y'], inplace=True)
cyclists_df.drop(columns=['cyclist'], inplace=True)


The sum is correct as we have as many entries as the rows in the dataset

In [ ]:
print(f"Null values in cyclist_experience column: {int(cyclists_df['cyclist_experience'].isnull().sum())}")

In [ ]:
cyclists_df['cyclist_experience'].describe()

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y='cyclist_experience', data=cyclists_df)
plt.title('Boxplot of Cylist Experience Column')
plt.ylabel('Best Position Standard Deviation')
plt.show()

In [ ]:
# Plot the distribution of the strength index
plt.figure(figsize=(8, 4))
sns.histplot(cyclists_df['cyclist_experience'], bins=100)
plt.title('Distribution of Best Cyclist Experience Column')
plt.xlabel('Best Position Standard Deviation')
plt.ylabel('Frequency')
plt.show()

In [ ]:
cyclists_df.value_counts('cyclist_experience')

#### ```km_per_year``` column

This column also represents the cyclist's overall experience. It highlights the total distance raced by the cyclist over the course of their career, calculated by aggregating the distances covered in all stages in which they have participated. The overall career span is defined as the interval between the initial and final stages of the cyclist's participation in these races.

In [ ]:
# First compute overall Km raced by the cyclist

# Group by cyclist and sum number km raced
# Note: aassuming no duplicates in same stage
km_raced = races_df.groupby('cyclist')["length"].sum().reset_index(name='overall_km')
km_raced["overall_km"] = km_raced["overall_km"] / 1000

# Merge the km_raced with cyclists_df to add the 'km_per_year' column
cyclists_df = cyclists_df.merge(km_raced, left_on='_url', right_on='cyclist', how='left')

# Replace NaN values in km_per_year with 0 (for cyclists with no participation records)
cyclists_df['overall_km'] = cyclists_df['overall_km'].fillna(0).astype(int)

# Check the output
print(cyclists_df[["_url", "overall_km"]].head(3))

cyclists_df.drop(columns=['cyclist'], inplace=True)


In [ ]:
# Verify correctness of the computation (sum of km_per_year should match number of rows in races_df)
print(f"Total km_per_year: {cyclists_df['overall_km'].sum()} computed on races_df")
print(f"Total km_per_year: {km_raced['overall_km'].sum()} computed on km_raced on cyclists_df")

In [ ]:
print(f"Null values in overall_km column: {int(cyclists_df['overall_km'].isnull().sum())}")

In [ ]:
# Count total yeras of career for each cyclist
career_span = races_df.groupby('cyclist')['start_date'].agg(['min', 'max'])
career_span['career_years'] = (pd.to_datetime(career_span['max']) - pd.to_datetime(career_span['min'])).dt.days + 1/ 365

# Merge the career_span with cyclists_df to add the 'career_years' column
cyclists_df = cyclists_df.merge(career_span[['career_years']], left_on='_url', right_index=True, how='left')

#print(cyclists_df.columns)

# Replace NaN values in career_years with 0 (for cyclists with no participation records)
cyclists_df['career_years'] = cyclists_df['career_years'].fillna(0).astype(float)

# Check the output
print(cyclists_df[["_url", "career_years"]].head(3))



In [ ]:
cyclists_df['career_years'].describe()

In [ ]:
cyclists_df['km_per_year'] = cyclists_df['overall_km'] / np.where(cyclists_df['career_years'] != 0, cyclists_df['career_years'], 1)
cyclists_df['km_per_year'] = cyclists_df['km_per_year'].round(2)

In [ ]:
cyclists_df['km_per_year'].describe()

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(total_races, bins=30)
plt.title('Distribution of Total Races')
plt.xlabel('Total Races')
plt.ylabel('Frequency')
plt.show()

### CORRELATION ANALYSIS


Cyclists correlation analysis


In [ ]:
from utils import center_and_scale, correlations, plot_correlations

normalized_cyclist, normalization_scalers_cyclist = center_and_scale(cyclists_df)
cyclists_corr = correlations(normalized_cyclist)

plot_correlations(cyclists_corr)

Races correlation analysis


In [ ]:
from utils import center_and_scale, correlations, plot_correlations

normalized_cyclist, normalization_scalers_cyclist = center_and_scale(races_df)
cyclists_corr = correlations(normalized_cyclist)

plot_correlations(cyclists_corr)